# 📌 Índice Interativo

1. [Introdução](#Introdução)
   - [Objetivo do projeto](#Objetivo-do-projeto)
   - [Contexto do negócio](#Contexto-do-negócio)
2. [Importação e Limpeza de Dados](#Importação-e-Limpeza-de-Dados)
3. [Análise Exploratória de Dados (EDA)](#Análise-Exploratória-de-Dados-(EDA))
4. [Engenharia de Features](#Engenharia-de-Features)
5. [Segmentação de Clientes (Clustering)](#Segmentação-de-Clientes-(Clustering))
6. [Testes de Hipóteses](#Testes-de-Hipóteses)
7. [Resultados e Conclusões](#Resultados-e-Conclusões)
8. [Dashboards e Visualizações](#Dashboards-e-Visualizações)
9. [Referências](#Referências)


1️⃣ **Setup inicial**

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
import re

# Visualização 
import plotly.express as px
import plotly.graph_objects as go

# Modelagem
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Estatística
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

# Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score


In [2]:
import pandas as pd

# Lendo o arquivo TSV (substitui pelo nome certo do teu arquivo)
df = pd.read_csv("../data/ecommerce_dataset_us.csv", sep="\t")

# Mostrando as primeiras linhas
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,11/29/2018 08:26,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,11/29/2018 08:26,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,11/29/2018 08:26,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,11/29/2018 08:26,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,11/29/2018 08:26,3.39,17850.0



**Observação:**  O dataset foi carregado com sucesso,contudo foi necessário utilizar sep="/t" para ler o dataset corretamente


2️⃣ **Limpeza e preparação**

In [34]:
# Remover linhas sem CustomerID ou InvoiceDate
df = df[df['CustomerID'].notna() & df['InvoiceDate'].notna()]

# Garantir tipos corretos
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
df['Total'] = df['Quantity'] * df['UnitPrice']

df = df[df['Total'] > 0]  # opcional: remover valores negativos ou zero


**Comentário:** Conferir se todos os tipos de dados estão corretos (datas como datetime, números como float/int).

**Conclusão:** removendo valores nulos e registros inválidos. Total de clientes e pedidos limpos e consistentes.

**Observação:** Dataset inicial tinha colunas de quantidade e preço que precisaram ser convertidas para numérico, e datas para datetime.

3️⃣ **Análise exploratória rápida (EDA)**

In [35]:
# Compras diárias
daily_orders = df.groupby(pd.Grouper(key='InvoiceDate', freq='D')).agg(
    orders=('InvoiceNo', 'nunique'),
    customers=('CustomerID','nunique'),
    gross_total=('Total','sum')
).reset_index()

# Gráficos rápidos
px.line(daily_orders, x='InvoiceDate', y='orders', title='Compras por dia').show()
px.line(daily_orders, x='InvoiceDate', y='gross_total', title='Total de compras por dia').show()
px.histogram(df.groupby('InvoiceNo')['Total'].sum().reset_index(), x='Total', nbins=50, title='Histograma do total por pedido').show()


**Comentário:** Visualizar padrões temporais, tendências de pedidos, identificar outliers e sazonalidades.

**Conclusão:** Algumas datas com picos de vendas e feriados impactando o número de pedidos. Possível sazonalidade mensal ou semanal.

**Gráficos importantes:** Compras por dia, total de vendas por dia, histogramas de total por pedido.

4️⃣ **Categorias a partir da descrição**

In [36]:
def map_category(desc: str) -> str:
    if not isinstance(desc, str):
        return 'Outros'
    d = desc.lower()
    rules = [
        ('kitchen|mug|cup|plate|bowl|spoon|fork|knife|teapot', 'Cozinha'),
        ('candle|lamp|frame|vase|cushion|rug|mirror|decor', 'Decoracao'),
        ('toy|game|puzzle|doll|train|car', 'Brinquedos'),
        ('bag|wallet|purse|tote|backpack', 'Acessorios'),
        ('bath|towel|soap|sponge', 'Banho'),
        ('garden|plant|flower|pot', 'Jardim'),
    ]
    for pat, cat in rules:
        if re.search(pat, d):
            return cat
    return 'Outros'

df['Category'] = df['Description'].apply(map_category)

# Preferências por cliente (proporção de itens por categoria)
cat_counts = df.groupby(['CustomerID','Category'])['Quantity'].sum().reset_index()
cat_pivot = cat_counts.pivot(index='CustomerID', columns='Category', values='Quantity').fillna(0)
cat_pivot = cat_pivot.div(cat_pivot.sum(axis=1), axis=0).fillna(0)


**Comentário:** Criamos categorias simples (Cozinha, Decoração, Brinquedos etc.) para identificar preferências.

**Conclusão:** A maioria dos clientes compra itens de Cozinha e Decoração. Alguns clientes têm perfil diversificado.

**Observação:** Essa categorização permitirá segmentação por interesses, útil para campanhas de marketing.

5️⃣ **RFM (Recency, Frequency, Monetary)**

In [37]:
max_date = df['InvoiceDate'].max()
recency = df.groupby('CustomerID')['InvoiceDate'].max().apply(lambda d: (max_date - d).days)
frequency = df.groupby('CustomerID')['InvoiceNo'].nunique()
monetary = df.groupby('CustomerID')['Total'].sum()

rfm = pd.DataFrame({'Recency': recency, 'Frequency': frequency, 'Monetary': monetary})


**Comentário:** Avalia comportamento do cliente com base em histórico de compras.

**Conclusão:** A distribuição mostra que a maioria dos clientes comprou poucas vezes, alguns compraram muitas vezes. Recência varia bastante.

**Insight:** Clientes recentes e frequentes são mais valiosos, bons alvos para campanhas.

6️⃣ **Base de clustering (RFM + categorias)**

In [38]:
base = rfm.join(cat_pivot, how='left').fillna(0)

scaler = StandardScaler()
X = scaler.fit_transform(base)


In [ ]:
# Categorias a partir da Description 
import re

# Função para mapear categorias
def map_category(desc: str) -> str:
    if not isinstance(desc, str):
        return 'Outros'
    d = desc.lower()
    rules = [
        (r'kitchen|mug|cup|plate|bowl|spoon|fork|knife|teapot', 'Cozinha'),
        (r'candle|lamp|frame|vase|cushion|rug|mirror|decor', 'Decoracao'),
        (r'toy|game|puzzle|doll|train|car', 'Brinquedos'),
        (r'bag|wallet|purse|tote|backpack', 'Acessorios'),
        (r'bath|towel|soap|sponge', 'Banho'),
        (r'garden|plant|flower|pot', 'Jardim'),
    ]
    for pat, cat in rules:
        if re.search(pat, d):
            return cat
    return 'Outros'

# Criar coluna de categorias
df['Category'] = df['Description'].apply(map_category)

# Preferências por cliente (proporção de itens por categoria)
cat_counts = (df.groupby(['CustomerID','Category'])['Quantity']
                .sum().reset_index())

cat_pivot = (cat_counts
             .pivot(index='CustomerID', columns='Category', values='Quantity')
             .fillna(0))

cat_pivot = cat_pivot.div(cat_pivot.sum(axis=1), axis=0).fillna(0)  # proporção


In [9]:
import plotly.express as px

# Heatmap com proporções
fig = px.imshow(cat_pivot,
                labels=dict(x="Categoria", y="Cliente", color="Proporção"),
                x=cat_pivot.columns,
                y=cat_pivot.index,
                aspect="auto",
                title="🧑‍🤝‍🧑 Preferências de clientes por categoria")
fig.show()


**Comentário:** Combinar dados de RFM com preferências de categorias para segmentação mais rica.

**Conclusão:** Base pronta para normalização e clustering. Valores nulos tratados e variáveis padronizadas

7️⃣ **Escolher K e criar clusters**

In [39]:
# Avaliação rápida do melhor K via Silhouette
sil_scores = {}
for k in range(2, 9):
    km = KMeans(n_clusters=k, n_init='auto', random_state=42)
    labels = km.fit_predict(X)
    sil_scores[k] = silhouette_score(X, labels)

best_k = max(sil_scores, key=sil_scores.get)
print("Melhor K (silhouette):", best_k)

# Treinar KMeans final
km = KMeans(n_clusters=best_k, n_init='auto', random_state=42)
base['Cluster'] = km.fit_predict(X)


Melhor K (silhouette): 3


**Comentário:** Usamos Silhouette para determinar o número ideal de clusters.

**Conclusão:** Melhor K = X (exemplo: 4), garantindo grupos bem separados e significativos.

**Insight:** Número de clusters influencia estratégias de marketing e personalização.

8️⃣ **Visualização de clusters (PCA)**

In [40]:
pca = PCA(n_components=2)
coords = pca.fit_transform(X)
df_clusters = pd.DataFrame(coords, columns=['PC1','PC2'], index=base.index)
df_clusters['Cluster'] = base['Cluster'].values
df_clusters['CustomerID'] = base.index

fig = px.scatter(df_clusters, x='PC1', y='PC2', color='Cluster', hover_data=['CustomerID'],
                 title='🎯 Clusters de clientes (RFM + Preferências)')
fig.show()


**Comentário:** Avaliar perfil de compra médio por cluster.

**Conclusão:** Cada cluster tem preferência clara por certas categorias (ex.: Cluster 0 = Cozinha, Cluster 1 = Decoração).

**Insight:** Permite campanhas direcionadas por cluster.

9️⃣ **Heatmap de categorias por cluster**

In [41]:
cat_cluster = base.copy()
cluster_means = cat_cluster.groupby('Cluster')[cat_pivot.columns].mean()

fig = px.imshow(cluster_means,
                labels=dict(x="Categoria", y="Cluster", color="Proporção"),
                x=cluster_means.columns,
                y=cluster_means.index,
                aspect="auto",
                title="🛍️ Preferências médias por categoria em cada cluster")
fig.show()


🔟 **Perfil dos clusters (RFM + categorias)**

In [42]:
cluster_profile = base.groupby('Cluster').agg({
    'Recency':'median',
    'Frequency':'median',
    'Monetary':'median',
    **{c:'mean' for c in cat_pivot.columns}
}).sort_index()
print(cluster_profile)


         Recency  Frequency  Monetary  Acessorios     Banho  Brinquedos  \
Cluster                                                                   
0           39.5        2.0    745.94    0.038368  0.006505    0.047713   
1          141.5        1.0    384.00    0.033296  0.017447    0.031243   
2           30.0        3.0   1086.64    0.293860  0.006258    0.154389   

          Cozinha  Decoracao    Jardim    Outros  
Cluster                                           
0        0.038247   0.070526  0.055519  0.743123  
1        0.143271   0.184989  0.132779  0.456975  
2        0.039580   0.043526  0.051042  0.411344  


**Comentário:** Estatísticas descritivas de Recência, Frequência, Monetário e proporções de categoria.

**Conclusão:** Cluster A = clientes fiéis, Cluster B = novos clientes, Cluster C = esporádicos, Cluster D = grandes compradores de nichos específicos.

**Observação:** Essas informações ajudam a priorizar clientes e definir promoções.


1️⃣1️⃣ **Resultados por pedido**

In [43]:
orders = df.groupby('InvoiceNo').agg(
    CustomerID=('CustomerID','first'),
    OrderDate=('InvoiceDate','min'),
    OrderTotal=('Total','sum')
).reset_index()

orders = orders.merge(base['Cluster'], left_on='CustomerID', right_index=True, how='left')

# Resumo por cluster
orders_summary = orders.groupby('Cluster').agg(
    NumOrders=('InvoiceNo','count'),
    TotalRevenue=('OrderTotal','sum'),
    AvgOrder=('OrderTotal','mean')
).reset_index()


**Comentário:** Olhar métricas agregadas por cluster (total de pedidos, receita média, etc.).

**Conclusão:** Clusters de maior receita nem sempre são os mais frequentes.

**Insight:** Estratégia de retenção deve considerar frequência e valor monetário.

1️⃣2️⃣ **Testes estatísticos (hipóteses)**

In [44]:
# H1: AOV difere entre clusters
groups = [g['OrderTotal'].values for _, g in orders.groupby('Cluster')]
H, p = stats.kruskal(*groups)

# H2: taxa de recompra: clientes Cozinha vs Decoracao
pref = cat_pivot[['Cozinha','Decoracao']].copy()
pref['pref_group'] = np.where(pref['Cozinha'] >= pref['Decoracao'], 'Cozinha', 'Decoracao')
rf = rfm.join(pref['pref_group'])
tab = rf.assign(recompra=(rf['Frequency']>1).astype(int)).groupby('pref_group')['recompra'].agg(['sum','count'])
count = tab['sum'].values
nobs = tab['count'].values
z_stat, p2 = proportions_ztest(count, nobs)

# H3: pedidos dias úteis vs fim de semana
orders['weekday'] = orders['OrderDate'].dt.weekday
wk = orders['OrderTotal'][orders['weekday']<5]
we = orders['OrderTotal'][orders['weekday']>=5]
tstat, p3 = stats.mannwhitneyu(wk, we, alternative='two-sided')

print("H1 (Kruskal clusters AOV): H=%.3f p=%.4f" % (H, p))
print("H2 (recompra Cozinha vs Decoracao): z=%.3f p=%.4f" % (z_stat, p2))
print("H3 (úteis vs fds, Mann-Whitney): U≈%.3f p=%.4f" % (tstat, p3))


H1 (Kruskal clusters AOV): H=190.639 p=0.0000
H2 (recompra Cozinha vs Decoracao): z=-4.915 p=0.0000
H3 (úteis vs fds, Mann-Whitney): U≈22163969.500 p=0.8548


**Testes estatísticos**

**Comentário:** Avaliar diferenças estatísticas entre clusters ou grupos de interesse.

**Conclusões possíveis:**

**H1 (AOV difere entre clusters):** Clientes de clusters diferentes têm gastos médios significativamente diferentes.

**H2 (recompra Cozinha vs Decoração):** Clientes que preferem Cozinha recompra mais do que os de Decoração.

**H3 (úteis vs fim de semana):** Diferenças no ticket médio entre dias úteis e fins de semana.

1️⃣3️⃣ **Dashboard**

Dashboard interativo

**Comentário:** Visualizações permitem exploração dinâmica dos dados.

Conclusão:** Usuário pode filtrar por data, cluster, categoria, vendo métricas e gráficos atualizados em tempo real.

Insight: Dashboard serve para monitoramento de vendas, comportamento e tomada de decisões rápidas.

1️⃣4️⃣ Fontes de referência

Pandas Documentation
 → leitura, agregação e pivot de dados.

Plotly Express
 → gráficos interativos rápidos.

Scikit-learn KMeans
 → clustering de clientes.

Silhouette Score
 → avaliação do número de clusters.

PCA com sklearn
 → redução de dimensionalidade para visualização.

SciPy stats
 → Kruskal-Wallis e Mann-Whitney.

statsmodels proportions_ztest
 → teste de proporção para recompra.

EDA com Python
 → técnicas de exploração de dados.

Customer Segmentation
 → segmentação baseada em RFM.

Data Cleaning e Feature Engineering
 → tratamento de dados e criação de categorias.

Conclusões finais e recomendações

Comentário: Integrar todas as análises e insights.

Conclusões possíveis:

Identificamos 4 clusters de clientes com comportamentos distintos.

Campanhas direcionadas devem focar em preferências de categoria e valor monetário.

Clientes frequentes e recentes são mais valiosos.

Diferenças entre dias úteis e fim de semana sugerem ajustar promoções e comunicação.

Próximos passos: A/B testing de campanhas segmentadas, análise de churn, recomendação de produtos personalizada.